In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
import statsmodels.formula.api as smf
from datetime import timedelta
from pandas.tseries.offsets import BMonthEnd, BusinessDay, MonthBegin
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection  import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_validate, GridSearchCV
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")
import scipy as sc
import seaborn as sns
from xgboost import XGBRegressor
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
random.seed(1)

In [2]:
# Use monthly data to avoid transaction costs
stocks = pd.read_feather('crsp_monthly.feather')
etf1 = stocks[(stocks['SHRCD']==73)]
#only work with stocks
stocks = stocks[(stocks['SHRCD']==10)|(stocks['SHRCD']==11)]
stocks['DATE'] = stocks['DATE'] + MonthEnd(0)
etf1['DATE'] = etf1['DATE'] + MonthEnd(0)
stocks['PRC']  = np.abs(stocks['PRC'])
stocks['MV'] = stocks['SHROUT']*stocks['PRC']
# stocks.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','VOL'], axis=1, inplace=True)
stocks.set_index(['PERMNO','DATE'], inplace=True)
stocks.sort_index(inplace=True)
stocks.head()

SHRCD  EXCHCD   SICCD       PRC     VOL       RET  SHROUT  \
PERMNO  DATE                                                                    
10000.0 1986-01-31   10.0     3.0  3990.0  4.375000  1771.0       NaN  3680.0   
        1986-02-28   10.0     3.0  3990.0  3.250000   828.0 -0.257143  3680.0   
        1986-03-31   10.0     3.0  3990.0  4.437500  1078.0  0.365385  3680.0   
        1986-04-30   10.0     3.0  3990.0  4.000000   957.0 -0.098592  3793.0   
        1986-05-31   10.0     3.0  3990.0  3.109375  1074.0 -0.222656  3793.0   

                              MV  
PERMNO  DATE                      
10000.0 1986-01-31  16100.000000  
        1986-02-28  11960.000000  
        1986-03-31  16330.000000  
        1986-04-30  15172.000000  
        1986-05-31  11793.859375

In [26]:
cstat = pd.read_feather('compustat.feather')
cstat.rename(columns={"LPERMNO":"PERMNO"}, inplace=True)
cstat['DATE'] = cstat['DATADATE'] + MonthEnd(0)
cstat.set_index(['PERMNO','DATE'], inplace=True)
cstat.sort_index(inplace=True)
cstat.head()

DATADATE   FYEAR      AT    CEQ      LT  PSTK    SEQ  \
PERMNO  DATE                                                                
10000.0 1986-10-31 1986-10-31  1986.0   2.115  0.418   1.697   0.0  0.418   
10001.0 1986-06-30 1986-06-30  1986.0  12.242  5.432   6.810   0.0  5.432   
        1987-06-30 1987-06-30  1987.0  11.771  5.369   6.402   0.0  5.369   
        1988-06-30 1988-06-30  1988.0  11.735  5.512   6.223   0.0  5.512   
        1989-06-30 1989-06-30  1989.0  18.565  6.321  12.244   0.0  6.321   

                       IB   CAPX  
PERMNO  DATE                      
10000.0 1986-10-31 -0.730  0.240  
10001.0 1986-06-30  0.669  0.551  
        1987-06-30  0.312  0.513  
        1988-06-30  0.542  0.240  
        1989-06-30  1.208  0.444

In [27]:
df = stocks.merge(cstat[['IB','SEQ']],how = 'left',on = ['PERMNO','DATE'])
df.head()

SHRCD  EXCHCD   SICCD       PRC     VOL       RET  SHROUT  \
PERMNO  DATE                                                                    
10000.0 1986-01-31   10.0     3.0  3990.0  4.375000  1771.0       NaN  3680.0   
        1986-02-28   10.0     3.0  3990.0  3.250000   828.0 -0.257143  3680.0   
        1986-03-31   10.0     3.0  3990.0  4.437500  1078.0  0.365385  3680.0   
        1986-04-30   10.0     3.0  3990.0  4.000000   957.0 -0.098592  3793.0   
        1986-05-31   10.0     3.0  3990.0  3.109375  1074.0 -0.222656  3793.0   

                              MV  IB  SEQ  
PERMNO  DATE                               
10000.0 1986-01-31  16100.000000 NaN  NaN  
        1986-02-28  11960.000000 NaN  NaN  
        1986-03-31  16330.000000 NaN  NaN  
        1986-04-30  15172.000000 NaN  NaN  
        1986-05-31  11793.859375 NaN  NaN

In [28]:
df=df.reset_index()
df=df.set_index(['PERMNO','DATE'])

In [29]:
# Shift by 3 months instead of 6. Assume you can get financial data every quarter
df['Lag_IB'] = df['IB'].groupby('PERMNO').shift(3).fillna(method = 'pad',limit = 15)
df['LAG_EP'] = df['Lag_IB']*1000/df['MV'].groupby('PERMNO').shift()
df['LAG_PRC'] = df['PRC'].groupby('PERMNO').shift()
df['LAG_VOL'] = df['VOL'].groupby('PERMNO').shift()
df['LAGRET'] = df['RET'].groupby(['PERMNO']).shift()

In [42]:
a = df[['RET']].groupby(['PERMNO']).std()

## Feature Engineering

### 1. Moving Average - past 5 months

In [43]:
df['MA']=df['LAGRET'].groupby(['PERMNO']).rolling(5).mean().droplevel(0)

### 2. Volatility and Moving_Volume

In [44]:
# 5 month volatility
df['VOLATILITY']=df['LAGRET'].groupby(['PERMNO']).rolling(5).std().droplevel(1) 
#create lag volume variable 
df['LAGVOL'] = df['VOL'].groupby(['PERMNO']).shift()
# 5 month moving volume
df['MOVING_VOL']=df['LAGVOL'].groupby(['PERMNO']).rolling(5).mean().droplevel(1) 

### 3. Reversal
Create a variable "C" to represent the number of consectuive positive return/negative return
- Example: C = -2 means the ETF has negative return for yesterday and today
- Example: C = 3 means the ETF has positive return for today, yesterday, and the day before yesterday

In [45]:
def compute_c(row):
    if row['RET']<0:
        if row['LAGRET']<0:
            return -1
        else:
            return 0
    else:
        if row['LAGRET'] >0:
            return 1
        else:
            return 0

In [46]:
cols = df.columns

In [47]:
# Code to populate column 'C'
# Keeps track of consecutive positive or negative returns
for etf, data in df.groupby(level=0):
    dataframe = data.droplevel(0)
    counter =0
    newc = []
    dataframe['temp']=dataframe.apply(compute_c,axis=1)
    v = dataframe['temp']
    grouper = (v!=v.shift()).cumsum()
    temporary=dataframe.groupby(grouper)['temp'].cumsum().reset_index()
    temporary['PERMNO'] = etf
    temporary=temporary.set_index(['PERMNO','DATE'])
    df.loc[df.index.isin([etf],level=0),'C'] = temporary['temp']

In [48]:
df = df.reset_index(level=0, drop=False)
df.head(10)

,PERMNO,SHRCD,EXCHCD,SICCD,PRC,VOL,RET,SHROUT,MV,IB,...,Lag_IB,LAG_EP,LAG_PRC,LAG_VOL,LAGRET,MA,VOLATILITY,LAGVOL,MOVING_VOL,C
DATE,,,,,,,,,,,,,,,,,,,,,
1986-01-31,10000.0,10.0,3.0,3990.0,4.375000,1771.0,NaN,3680.0,16100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1986-02-28,10000.0,10.0,3.0,3990.0,3.250000,828.0,-0.257143,3680.0,11960.000000,NaN,...,NaN,NaN,4.375000,1771.0,NaN,NaN,NaN,1771.0,NaN,0.0
1986-03-31,10000.0,10.0,3.0,3990.0,4.437500,1078.0,0.365385,3680.0,16330.000000,NaN,...,NaN,NaN,3.250000,828.0,-0.257143,NaN,NaN,828.0,NaN,0.0
1986-04-30,10000.0,10.0,3.0,3990.0,4.000000,957.0,-0.098592,3793.0,15172.000000,NaN,...,NaN,NaN,4.437500,1078.0,0.365385,NaN,NaN,1078.0,NaN,0.0
1986-05-31,10000.0,10.0,3.0,3990.0,3.109375,1074.0,-0.222656,3793.0,11793.859375,NaN,...,NaN,NaN,4.000000,957.0,-0.098592,NaN,NaN,957.0,NaN,-1.0
1986-06-30,10000.0,10.0,3.0,3990.0,3.093750,1069.0,-0.005025,3793.0,11734.593750,NaN,...,NaN,NaN,3.109375,1074.0,-0.222656,NaN,NaN,1074.0,1141.6,-2.0
1986-07-31,10000.0,10.0,3.0,3990.0,2.843750,1163.0,-0.080808,3793.0,10786.343750,NaN,...,NaN,NaN,3.093750,1069.0,-0.005025,-0.043606,0.249721,1069.0,1001.2,-3.0
1986-08-31,10000.0,10.0,3.0,3990.0,1.093750,3049.0,-0.615385,3793.0,4148.593750,NaN,...,NaN,NaN,2.843750,1163.0,-0.080808,-0.008339,0.223052,1163.0,1068.2,-4.0
1986-09-30,10000.0,10.0,3.0,3990.0,1.031250,3551.0,-0.057143,3793.0,3911.531250,NaN,...,NaN,NaN,1.093750,3049.0,-0.615385,-0.204493,0.242622,3049.0,1462.4,-5.0


### 4. Momentum

In [49]:
def calculate_momentum(df):
    return (df['LAGRET'].rolling(12).sum() - df['LAGRET']) *(1/11)

In [50]:
df['MOMENTUM']=df.groupby(['PERMNO']).apply(calculate_momentum).droplevel(0)
df.head()

,PERMNO,SHRCD,EXCHCD,SICCD,PRC,VOL,RET,SHROUT,MV,IB,...,LAG_EP,LAG_PRC,LAG_VOL,LAGRET,MA,VOLATILITY,LAGVOL,MOVING_VOL,C,MOMENTUM
DATE,,,,,,,,,,,,,,,,,,,,,
1986-01-31,10000.0,10.0,3.0,3990.0,4.375000,1771.0,NaN,3680.0,16100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1986-02-28,10000.0,10.0,3.0,3990.0,3.250000,828.0,-0.257143,3680.0,11960.000000,NaN,...,NaN,4.3750,1771.0,NaN,NaN,NaN,1771.0,NaN,0.0,NaN
1986-03-31,10000.0,10.0,3.0,3990.0,4.437500,1078.0,0.365385,3680.0,16330.000000,NaN,...,NaN,3.2500,828.0,-0.257143,NaN,NaN,828.0,NaN,0.0,NaN
1986-04-30,10000.0,10.0,3.0,3990.0,4.000000,957.0,-0.098592,3793.0,15172.000000,NaN,...,NaN,4.4375,1078.0,0.365385,NaN,NaN,1078.0,NaN,0.0,NaN
1986-05-31,10000.0,10.0,3.0,3990.0,3.109375,1074.0,-0.222656,3793.0,11793.859375,NaN,...,NaN,4.0000,957.0,-0.098592,NaN,NaN,957.0,NaN,-1.0,NaN


## Strategy Exploration

### Machine Learning - Neural Net (Data Leak, Does not work)

In [51]:
df1 = df.copy()
df1.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','IB','SEQ','VOL'], axis=1,inplace=True)
df1 = df1.dropna()

In [52]:
# Training set time range: 2000-2020
# df_train = df1.reset_index(level=0, drop=False)
df_train = df1[(df1.index<='2020-12-31')&(df1.index>='2000-01-01')]
df_train=df_train.reset_index()
df_train=df_train.set_index(['PERMNO','DATE'])

# # Validation set time range: time stamp after 2020-12-31
# df_test = df1.reset_index(level=0, drop=False)
df_test = df1[(df1.index>'2020-12-31')]
df_test=df_test.reset_index()
df_test=df_test.set_index(['PERMNO','DATE'])

In [53]:
X_train = df_train.drop(['RET'],axis=1)
y_train = df_train['RET']
X_train = (X_train - X_train.mean())/(X_train.std())
y_train=y_train.ravel()
y_train =y_train.reshape(-1,1)
yscaler =StandardScaler().fit(y_train)
y_train=yscaler.transform(y_train)

In [54]:
X_test = df_test.drop(['RET'],axis=1)
y_test = df_test['RET']
X_test = (X_test - X_test.mean())/(X_test.std())
y_test=y_test.ravel()
y_test =y_test.reshape(-1,1)
yscaler =StandardScaler().fit(y_test)
y_test=yscaler.transform(y_test)

In [55]:
NN = MLPRegressor(hidden_layer_sizes=(2,4), solver='adam') 
NN.fit(X_train,y_train.ravel())
y_pred = NN.predict(X_train)

In [56]:
y_pred = yscaler.inverse_transform(y_pred.reshape(-1,1))
print(f'R-squared is {r2_score(df_train["RET"],y_pred)}')

R-squared is 0.39590281719607723


In [57]:
df_train['pred'] = y_pred
df_train['pred_q'] = pd.qcut(df_train['pred'],q=5,labels=range(1,6))
test = df_train.copy()
a = test[['pred']].groupby(['PERMNO']).std()
a['volatility_q'] = pd.qcut(a['pred'],q=5,labels=range(1,6))

In [58]:
test = test.reset_index()
df_new = test[test['PERMNO'].isin(a[a['volatility_q']>4].index)]

In [59]:
RF = 0.03/12
# portfolio = df_new.groupby(['pred_q','DATE'])[['RET']].mean()
# highminuslow = portfolio.loc[5] - portfolio.loc[1]
# stats = highminuslow.describe()
# stats.loc['Sharpe'] = (stats.loc['mean'] - RF)/stats.loc['std']*(12**0.5)
# se = stats['RET']['std']/np.sqrt(stats['RET']['count'])
# print(f"t-stats: {round(stats['RET']['mean']/se,2)}")
# stats

In [60]:
y_pred1 = NN.predict(X_test)
y_pred1 = yscaler.inverse_transform(y_pred1.reshape(-1,1))
print(f'R-squared is {r2_score(df_test["RET"],y_pred1)}')

R-squared is 0.061484422215301704


#### 100% ML (Data Leak, does not work)

In [61]:
df_test['pred'] = y_pred1
df_test['pred_q'] = pd.qcut(df_test['pred'],q=4,labels=range(1,5))
test1 = df_test.copy()
# b = test1[['pred']].groupby(['PERMNO']).std()
# b['volatility_q'] = pd.qcut(a['pred'],q=5,labels=range(1,6))

In [62]:
test1 = test1.reset_index()
# df_new1 = test1[test1['PERMNO'].isin(b[b['volatility_q']>4].index)]
portfolio1 = test1.groupby(['pred_q','DATE'])[['RET']].mean()
highminuslow1 = portfolio1.loc[4] - portfolio1.loc[1]
stats1 = highminuslow1.describe()
stats1.loc['Sharpe'] = (stats1.loc['mean'] - RF)/stats1.loc['std']*(12**0.5)
se1 = stats1['RET']['std']/np.sqrt(stats1['RET']['count'])
print(f"t-stats: {round(stats1['RET']['mean']/se1,2)}")
stats1

t-stats: 11.34


,RET
count,18.000000
mean,0.189103
std,0.070725
min,0.054331
25%,0.152270
50%,0.197341
75%,0.221255
max,0.341832
Sharpe,9.139736


#### ML + Momentum

In [63]:
def quintiles(inser):
    outser = pd.qcut(inser, q=5, labels=range(1,6))
    return outser

In [64]:
df_test['M_Q'] = df_test.groupby('DATE')['MOMENTUM'].apply(quintiles)
portfolio_m = df_test.groupby(['M_Q','DATE'])[['RET']].mean()
highminuslow2 = portfolio_m.loc[5] - portfolio_m.loc[1]

In [65]:
for m in range(0,11,1):
        final=(m*0.1)* highminuslow1 + ((10-m)*0.1)* highminuslow2
        stats1 = final.describe()
        stats1.loc['Sharpe'] = (stats1.loc['mean'] - RF)/stats1.loc['std']*(12**0.5)
        se1 = stats1['RET']['std']/np.sqrt(stats1['RET']['count'])
        print(f"{round(m*10,2)}% ML + {round((10-m)*10,2)}% Momentum : Mean is {round(stats1['RET']['mean'],4)}, Sharpe is {round(stats1['RET']['Sharpe'],4)}")

0% ML + 100% Momentum : Mean is 0.0204, Sharpe is 1.1804
10% ML + 90% Momentum : Mean is 0.0373, Sharpe is 2.5112
20% ML + 80% Momentum : Mean is 0.0541, Sharpe is 4.0069
30% ML + 70% Momentum : Mean is 0.071, Sharpe is 5.5364
40% ML + 60% Momentum : Mean is 0.0879, Sharpe is 6.9066
50% ML + 50% Momentum : Mean is 0.1048, Sharpe is 7.9561
60% ML + 40% Momentum : Mean is 0.1216, Sharpe is 8.6378
70% ML + 30% Momentum : Mean is 0.1385, Sharpe is 9.0091
80% ML + 20% Momentum : Mean is 0.1554, Sharpe is 9.1649
90% ML + 10% Momentum : Mean is 0.1722, Sharpe is 9.189
100% ML + 0% Momentum : Mean is 0.1891, Sharpe is 9.1397


In [66]:
final = highminuslow1*0.3+highminuslow2*0.7
stats2 = final.describe()
stats2.loc['Sharpe'] = (stats1.loc['mean'] - RF)/stats2.loc['std']*(12**0.5)
se2 = stats2['RET']['std']/np.sqrt(stats2['RET']['count'])
print(f"t-stats: {round(stats2['RET']['mean']/se2,2)}")
stats2

t-stats: 7.03


,RET
count,18.000000
mean,0.071013
std,0.042868
min,-0.002819
25%,0.049679
50%,0.071375
75%,0.088253
max,0.163548
Sharpe,15.079054


### Machine Learning - Xgboost

In [67]:
df2 = df.copy()
df2.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','IB','SEQ','VOL'], axis=1,inplace=True)
df2 = df2.dropna()

In [68]:
# Training set time range: 2000-2020
df_train2 = df2[(df2.index<='2020-12-31')&(df2.index>='2000-01-01')]
df_train2=df_train2.reset_index()
df_train2=df_train2.set_index(['PERMNO','DATE'])

# # Validation set time range: time stamp after 2020-12-31
df_test2 = df2[(df2.index>'2020-12-31')]
df_test2=df_test2.reset_index()
df_test2=df_test2.set_index(['PERMNO','DATE'])

In [69]:
X_train2 = df_train2.drop(['RET'],axis=1)
y_train2 = df_train2['RET']
X_train2 = (X_train2 - X_train2.mean())/(X_train2.std())
y_train2=y_train2.ravel()
y_train2 =y_train2.reshape(-1,1)
yscaler =StandardScaler().fit(y_train2)
y_train2=yscaler.transform(y_train2)

In [70]:
X_test2 = df_test2.drop(['RET'],axis=1)
y_test2 = df_test2['RET']
X_test2 = (X_test2 - X_test2.mean())/(X_test2.std())
y_test2=y_test2.ravel()
y_test2 =y_test2.reshape(-1,1)
yscaler =StandardScaler().fit(y_test2)
y_test2=yscaler.transform(y_test2)

In [71]:
xgb = XGBRegressor(eta=0.025,max_depth=5, subsample=0.8, colsample_bytree=0.1)
xgb.fit(X_train2,y_train2.ravel())

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.025,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.0250000004,
             max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', ...)

In [72]:
y_pred2 = xgb.predict(X_train2)
y_pred2 = yscaler.inverse_transform(y_pred2.reshape(-1,1))
print(f'R-squared for training is {r2_score(df_train2["RET"],y_pred2)}')

R-squared for training is 0.09554264948461999


In [73]:
y_pred3 = xgb.predict(X_test2)
y_pred3 = yscaler.inverse_transform(y_pred3.reshape(-1,1))
print(f'R-squared for testing is {r2_score(df_test2["RET"],y_pred3)}')

R-squared for testing is -0.042476956858357395


In [74]:
df_test2['pred'] = y_pred3
df_test2['pred_q'] = pd.qcut(df_test2['pred'],q=5,labels=range(1,6))
test2 = df_test2.copy()
b2 = test2[['pred']].groupby(['PERMNO']).std()
b2['volatility_q'] = pd.qcut(b2['pred'],q=5,labels=range(1,6))

In [75]:
# Portfolio with ML strategy
test2 = test2.reset_index()
df_new2 = test2[test2['PERMNO'].isin(b2[b2['volatility_q']>4].index)]
portfolio3 = df_new2.groupby(['pred_q','DATE'])[['RET']].mean()
highminuslow3 = portfolio3.loc[5] - portfolio3.loc[1]

# Portfolio with MA strategy: Moving average 1 months
df_new3=df_new2.set_index(['PERMNO','DATE'])
df_new3['MA1']=df_new3['LAGRET'].groupby(['PERMNO']).rolling(1).mean().droplevel(0)
df_new3=df_new3.loc[df_new3['pred'] >= df_new3['MA1']]
portfolio4 = df_new3.groupby(['pred_q','DATE'])[['RET']].mean()
highminuslow4 = portfolio4.loc[5] - portfolio4.loc[1]

# Test with threshold
RF = 0.03/12  
for m in range(0,11,1):
        final=(m*0.1)* highminuslow3 + ((10-m)*0.1)* highminuslow4
        stats1 = final.describe()
        stats1.loc['Sharpe'] = (stats1.loc['mean'] - RF)/stats1.loc['std']*(12**0.5)
        se1 = stats1['RET']['std']/np.sqrt(stats1['RET']['count'])
        print(f"{round(m*10,2)} ML + {round((10-m)*10,2)}% MA : Mean is {round(stats1['RET']['mean'],4)}, Sharpe is {round(stats1['RET']['Sharpe'],4)}")

0 ML + 100% MA : Mean is 0.1169, Sharpe is 4.756
10 ML + 90% MA : Mean is 0.119, Sharpe is 4.7508
20 ML + 80% MA : Mean is 0.1212, Sharpe is 4.7401
30 ML + 70% MA : Mean is 0.1233, Sharpe is 4.7247
40 ML + 60% MA : Mean is 0.1254, Sharpe is 4.7052
50 ML + 50% MA : Mean is 0.1275, Sharpe is 4.6822
60 ML + 40% MA : Mean is 0.1297, Sharpe is 4.6562
70 ML + 30% MA : Mean is 0.1318, Sharpe is 4.6277
80 ML + 20% MA : Mean is 0.1339, Sharpe is 4.5973
90 ML + 10% MA : Mean is 0.136, Sharpe is 4.5652
100 ML + 0% MA : Mean is 0.1381, Sharpe is 4.5319


In [76]:
# Result:
final2=(1)* highminuslow3 + (0)* highminuslow4
stats3 = final2.describe()
stats3.loc['Sharpe'] = (stats3.loc['mean'] - RF)/stats3.loc['std']*(12**0.5)
se3 = stats3['RET']['std']/np.sqrt(stats3['RET']['count'])
print(f"t-stats: {round(stats3['RET']['mean']/se3,2)}")
stats3

t-stats: 5.65


,RET
count,18.000000
mean,0.138149
std,0.103687
min,-0.052486
25%,0.090326
50%,0.112263
75%,0.157693
max,0.423694
Sharpe,4.531938


### Machine Learning - Rolling Prediction (FINAL STRATEGY. NO DATA LEAKS)

In [5]:
df.to_csv('data.csv')
df = pd.read_csv('data.csv',parse_dates=[0],index_col=[0])

In [6]:
df = df.loc['2000-01-01':]

In [7]:
df['lagC'] = df['C'].shift(1)
df['lagMV'] = df['MV'].shift(1)

In [8]:
#df.reset_index(inplace=True)
df2 = df.copy()
df2.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','IB','SEQ','VOL','C','MV'], axis=1,inplace=True)
df2 = df2.dropna()

In [9]:
train = df2.loc[:'2010'] # 2000 to 2010 inclusive
test = df2.loc['2011':'2022'] # 2011 to 2022 
date_index = test.index.unique()

In [11]:
from lightgbm import LGBMRegressor

In [12]:
for i,train_end in enumerate(date_index):
    if i == len(date_index)-1:
        break 
    else:
        ts=train_end - MonthBegin()
        ts-= timedelta(days=1)
        ts = ts.strftime('%Y-%m-%d')
        X_train = df2.loc[:ts].drop(['RET'],axis=1)
        y_train = df2.loc[:ts,'RET']
        # Use 2 models
        # LGBM
        model = LGBMRegressor() # simple, fast nonlinear model
        M=model.fit(X_train,y_train)
        # Model 2
        xgb = XGBRegressor()
        M1=xgb.fit(X_train,y_train)

        # Rolling Prediction 
        future = train_end.strftime('%Y-%m-%d')
        X = df2.loc[future].drop(['RET'],axis=1)

        # Predict using LGBM
        df2.loc[future,'LGpreds']= M.predict(X)

        #Predict using XGBoost
        df2.loc[future,'XGpreds']= M1.predict(X)

    # clear_output()

In [59]:
# df2=df2.dropna()

In [17]:
df2['preds'] = df2[['XGpreds', 'LGpreds']].mean(axis=1)

In [13]:
df2.to_csv('latest2.csv')

In [89]:
### Now that we have predicted returns, let us move on to the strategy
# Assume we start investing from 2011

In [90]:
# Invest if predicted return is greater than 5MA month MA
# Cash if predicted return is lesser than 5 month MA
# 20%

# Long on High Momentum stocks, Short on Low Momentum stocks. High - low portfolio
# 80% weight

In [14]:
df2=df2.reset_index().dropna()

In [15]:
df2=df2.set_index(['DATE']).sort_index()

In [24]:
from sklearn.metrics import r2_score

r2 = r2_score(df2['RET'],df2['LGpreds'])

In [25]:
r2

-0.0034605981265320995

### Final Strategy: ML + Momentum

In [3]:
df_final=pd.read_csv("latest1.csv")
df_final.head()

,DATE,PERMNO,RET,Lag_IB,LAG_EP,LAG_PRC,LAG_VOL,LAGRET,MA,VOLATILITY,LAGVOL,MOVING_VOL,MOMENTUM,lagC,lagMV,LGpreds,XGpreds
0,2000-02-29,10001.0,0.015385,1.587,0.079724,8.12500,403.0,-0.044118,-0.007593,0.059703,403.0,456.8,-0.005199,-2.0,19906.25,NaN,NaN
1,2000-03-31,10001.0,-0.015758,1.587,0.078516,8.25000,222.0,0.015385,0.009884,0.047724,222.0,352.0,-0.000945,0.0,20212.50,NaN,NaN
2,2000-04-30,10001.0,0.011719,1.587,0.080509,8.00000,723.0,-0.015758,-0.010455,0.021882,723.0,425.0,-0.000741,0.0,19712.00,NaN,NaN
3,2000-05-31,10001.0,-0.023166,1.587,0.079577,8.09375,263.0,0.011719,-0.007392,0.024047,263.0,387.2,-0.003473,0.0,19943.00,NaN,NaN
4,2000-06-30,10001.0,0.027668,1.587,0.081464,7.90625,221.0,-0.023166,-0.011188,0.024897,221.0,366.4,-0.000487,0.0,19481.00,NaN,NaN


In [4]:
df_final=df_final.dropna()
df_final=df_final.set_index(['DATE','PERMNO']).sort_index()
df_final1 = df_final.copy()

In [5]:
df_final

RET    Lag_IB    LAG_EP     LAG_PRC    LAG_VOL  \
DATE       PERMNO                                                         
2011-01-31 10001.0  0.028992     6.819  0.082741   10.520000     5200.0   
           10002.0  0.041198  -121.295 -2.575477    2.670000     7350.0   
           10025.0  0.067052    -0.523 -0.003280   25.950001     3083.0   
           10026.0 -0.119610    48.409  0.054121   48.240002    11440.0   
           10028.0  0.032710     0.481  0.011428    4.280000     1775.0   
...                      ...       ...       ...         ...        ...   
2022-05-31 93423.0 -0.233081   129.923  0.039363   38.270000   261806.0   
           93426.0 -0.025953    20.221  0.051335   31.209999     7203.0   
           93429.0 -0.001682   529.000  0.044101  112.980003   138741.0   
           93434.0 -0.079710   -19.171 -0.334417    1.380000    44321.0   
           93436.0 -0.129197  5519.000  0.006118  870.760010  5046142.0   

                      LAGRET        MA  VOLATILITY     LAGVOL  MOVING_VOL  \
DATE       PERMNO                                                           
2011-01-31 10001.0  0.052291 -0.013607    0.093746     5200.0      4776.8   
           10002.0  0.063745 -0.025364    0.072135     7350.0      4922.2   
           10025.0  0.079002 -0.017061    0.098703     3083.0      4750.4   
           10026.0  0.056995  0.033046    0.078609    11440.0     10864.8   
           10028.0  0.043903  0.090600    0.168911     1775.0      3229.2   
...                      ...       ...         ...        ...         ...   
2022-05-31 93423.0 -0.120230  0.014695    0.119179   261806.0    302557.2   
           93426.0 -0.029238 -0.016178    0.080442     7203.0      8198.6   
           93429.0 -0.012585 -0.024634    0.039301   138741.0    127605.2   
           93434.0 -0.352113 -0.129980    0.134011    44321.0     52875.2   
           93436.0 -0.191945 -0.043034    0.164357  5046142.0   5367011.2   

                    MOMENTUM  lagC         lagMV   LGpreds   XGpreds  
DATE       PERMNO                                                     
2011-01-31 10001.0  0.004532   0.0  8.241368e+04  0.019102  0.018099  
           10002.0  0.005313   0.0  4.709613e+04  0.022959 -0.080367  
           10025.0 -0.034111   0.0  1.594368e+05  0.003008 -0.001898  
           10026.0  0.014772   3.0  8.944661e+05  0.009553  0.010078  
           10028.0  0.113495   3.0  4.208952e+04  0.024555  0.022364  
...                      ...   ...           ...       ...       ...  
2022-05-31 93423.0 -0.004266  -1.0  3.300596e+06  0.003116  0.004289  
           93426.0  0.002629   0.0  3.939014e+05  0.007616  0.010400  
           93429.0  0.011042  -3.0  1.199531e+07  0.005822  0.009778  
           93434.0 -0.028682  -5.0  5.732658e+04 -0.009721  0.255512  
           93436.0  0.049548   0.0  9.021161e+08 -0.019350  0.056581  

[471132 rows x 15 columns]

In [6]:
# Portfolio with ML strategy: 
# 1. Filtering data: only look at 
# 2. Quintile on predicted retuns and choose high
df_final['MA_q']= df_final['LGpreds'].groupby(['DATE']).apply(lambda x: pd.qcut(x,q=5,labels=range(1,6)))
portfolio4 = df_final.groupby(['MA_q','DATE'])[['RET']].mean()
high1 = portfolio4.loc[5]
low1 = portfolio4.loc[1]
port1 = high1-low1

# Portfolio with momentum strategy
# 1. Quintile on momentum and choose high
df_final['M_Q'] = df_final['MOMENTUM'].groupby(['DATE']).apply(lambda x: pd.qcut(x,q=5,labels=range(1,6)))
portfolio5= df_final.groupby(['M_Q','DATE'])[['RET']].mean()
high2 = portfolio5.loc[5] 
low2 = portfolio5.loc[1] 
port2 = high2-low2


# Test with threshold
RF = 0.03/12  
for m in range(0,11,1):
        final=(m*0.1)* port1 + ((10-m)*0.1)* port2
        stats1 = final.describe()
        stats1.loc['Sharpe'] = (stats1.loc['mean'] - RF)/stats1.loc['std']*(12**0.5)
        se1 = stats1['RET']['std']/np.sqrt(stats1['RET']['count'])
        print(f"{round(m*10,2)}% ML + {round((10-m)*10,2)}% Momentum : Mean is {round(stats1['RET']['mean'],4)}, Sharpe is {round(stats1['RET']['Sharpe'],4)}")

0% ML + 100% Momentum : Mean is 0.0051, Sharpe is 0.2117
10% ML + 90% Momentum : Mean is 0.0061, Sharpe is 0.3276
20% ML + 80% Momentum : Mean is 0.0071, Sharpe is 0.4674
30% ML + 70% Momentum : Mean is 0.0082, Sharpe is 0.6352
40% ML + 60% Momentum : Mean is 0.0092, Sharpe is 0.8324
50% ML + 50% Momentum : Mean is 0.0102, Sharpe is 1.0541
60% ML + 40% Momentum : Mean is 0.0113, Sharpe is 1.2826
70% ML + 30% Momentum : Mean is 0.0123, Sharpe is 1.4866
80% ML + 20% Momentum : Mean is 0.0133, Sharpe is 1.6323
90% ML + 10% Momentum : Mean is 0.0143, Sharpe is 1.704
100% ML + 0% Momentum : Mean is 0.0154, Sharpe is 1.7117


### Backtest

In [7]:
#benchmark1 : buy and hold stocks
stocks = stocks.reset_index()
benchmark_s = stocks[(stocks['DATE']>='2011-01-31')&(stocks['DATE']<='2022-12-31')].groupby('DATE').mean()['RET']
stat_bs = benchmark_s.describe()
benchmark_s = pd.DataFrame(benchmark_s)
stat_bs['sharpe']= stat_bs.loc['mean']/stat_bs.loc['std']*np.sqrt(12)
stat_bs['t-test']= stat_bs.loc['mean'] / stat_bs.loc['std'] * np.sqrt(stat_bs.loc['count'])
stat_bs

count     138.000000
mean        0.009055
std         0.056338
min        -0.223813
25%        -0.017831
50%         0.010659
75%         0.039548
max         0.205248
sharpe      0.556758
t-test      1.888060
Name: RET, dtype: float64

In [8]:
#benchmark2 : buy and hold etfs
etf1 = etf1.reset_index()
benchmark_etf = etf1[(etf1['DATE']>='2011-01-31')&(etf1['DATE']<='2022-12-31')].groupby('DATE').mean()['RET']
stat_be=benchmark_etf.describe()
benchmark_etf = pd.DataFrame(benchmark_etf)
stat_be['sharpe']= stat_be.loc['mean']/stat_be.loc['std']*np.sqrt(12)
stat_be['t-test']= stat_be.loc['mean'] / stat_be.loc['std'] * np.sqrt(stat_be.loc['count'])
stat_be

count     138.000000
mean        0.003767
std         0.031682
min        -0.142762
25%        -0.010625
50%         0.006672
75%         0.020208
max         0.100034
sharpe      0.411862
t-test      1.396692
Name: RET, dtype: float64

In [9]:
momentum = high2
stats_bm = momentum.describe()
stats_bm.loc['Sharpe'] = (stats_bm.loc['mean'] - RF)/stats_bm.loc['std']*(12**0.5)
se_bm = stats_bm['RET']['std']/np.sqrt(stats_bm['RET']['count'])
print(f"t-stats: {round(stats_bm['RET']['mean']/se_bm,2)}")
stats_bm

t-stats: 2.22


,RET
count,137.000000
mean,0.011704
std,0.061650
min,-0.208805
25%,-0.016021
50%,0.011773
75%,0.042239
max,0.222944
Sharpe,0.517148


In [10]:
ml = port1
stats_ml = ml.describe()
stats_ml.loc['Sharpe'] = (stats_ml.loc['mean'] - RF)/stats_ml.loc['std']*(12**0.5)
se_ml = stats_ml['RET']['std']/np.sqrt(stats_ml['RET']['count'])
print(f"t-stats: {round(stats_ml['RET']['mean']/se_ml,2)}")
stats_ml

t-stats: 6.91


,RET
count,137.000000
mean,0.015380
std,0.026065
min,-0.043633
25%,-0.003428
50%,0.013178
75%,0.029129
max,0.099102
Sharpe,1.711741


In [11]:
final = 0.5 * port1 + 0.5* port2
stats_bf = final.describe()
stats_bf.loc['Sharpe'] = (stats_bf.loc['mean'] - RF)/stats_bf.loc['std']*(12**0.5)
se_bf = stats_bf['RET']['std']/np.sqrt(stats_bf['RET']['count'])
print(f"t-stats: {round(stats_bf['RET']['mean']/se_bf,2)}")
stats_bf

t-stats: 4.71


,RET
count,137.000000
mean,0.010222
std,0.025378
min,-0.070953
25%,-0.002078
50%,0.009851
75%,0.024779
max,0.091284
Sharpe,1.054071


In [12]:
#Time Series plot for our strategy
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=final.index, y=final['RET'],name="Strategy"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=benchmark_etf.index, y=benchmark_etf['RET'],name="Benchmark - ETF"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=benchmark_s.index, y=benchmark_s['RET'],name="Benchmark - Stocks"),
    secondary_y=False,
)
fig.show()

In [13]:
#Time Series plot for Momentum
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=momentum.index, y=momentum['RET'],name="Momentum"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=benchmark_etf.index, y=benchmark_etf['RET'],name="Benchmark - ETF"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=benchmark_s.index, y=benchmark_s['RET'],name="Benchmark - Stocks"),
    secondary_y=False,
)
fig.show()

In [14]:
#Time Series plot for Momentum vs. Our Strategy
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=momentum.index, y=momentum['RET'],name="Momentum"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=final.index, y=final['RET'],name="Strategy"),
    secondary_y=False,
)
fig.show()

In [15]:
#Dollar return
benchmark_etf = benchmark_etf.reset_index()
benchmark_s = benchmark_s.reset_index()
final = final.reset_index()
final['DATE'] = pd.to_datetime(final['DATE'])
momentum = momentum.reset_index()
momentum['DATE'] = pd.to_datetime(momentum['DATE'])
value1 = pd.merge(benchmark_etf,final,how='outer',on = 'DATE',suffixes=['_etf','_strategy'])
value2 = pd.merge(benchmark_s,momentum,how='outer',on = 'DATE',suffixes=['_stock','_momentum'])
value = pd.merge(value1,value2,how='outer',on = 'DATE')
value.drop(value.tail(1).index,inplace=True) # drop last n rows

In [16]:
value

,DATE,RET_etf,RET_strategy,RET_stock,RET_momentum
0,2011-01-31,0.003763,0.003700,0.019941,0.013382
1,2011-02-28,0.021224,0.015457,0.044153,0.054916
2,2011-03-31,0.007983,0.024598,0.012816,0.032709
3,2011-04-30,0.025879,0.005687,0.015235,0.016043
4,2011-05-31,-0.015228,0.003431,-0.020456,-0.024300
...,...,...,...,...,...
132,2022-01-31,-0.041639,0.061885,-0.089647,-0.101862
133,2022-02-28,-0.011971,0.044962,-0.006627,0.022790
134,2022-03-31,0.008566,0.014938,0.017573,0.032637
135,2022-04-30,-0.063148,0.091284,-0.107631,-0.081475


In [17]:
value = value.set_index('DATE')
returns = value*100
returns = returns.cumsum()
returns

,RET_etf,RET_strategy,RET_stock,RET_momentum
DATE,,,,
2011-01-31,0.376340,0.370025,1.994118,1.338189
2011-02-28,2.498697,1.915744,6.409459,6.829831
2011-03-31,3.296983,4.375555,7.691094,10.100712
2011-04-30,5.884881,4.944280,9.214634,11.704968
2011-05-31,4.362054,5.287426,7.169042,9.274920
...,...,...,...,...
2022-01-31,65.234371,118.354624,144.007999,162.462933
2022-02-28,64.037247,122.850830,143.345296,164.741929
2022-03-31,64.893875,124.344627,145.102600,168.005585


In [18]:
portfolio_rets=pd.DataFrame()

In [19]:
portfolio_rets['RET_strategy']=returns['RET_strategy']
portfolio_rets['RET_strategy']+=100

In [20]:
portfolio_rets

,RET_strategy
DATE,
2011-01-31,100.370025
2011-02-28,101.915744
2011-03-31,104.375555
2011-04-30,104.944280
2011-05-31,105.287426
...,...
2022-01-31,218.354624
2022-02-28,222.850830
2022-03-31,224.344627


In [21]:
returns = value*100
returns = ((1 + value).cumprod()) *100
returns = returns.reset_index()
#returns = returns.drop(['RET_strategy','RET_momentum'],axis=1)

In [22]:
returns

,DATE,RET_etf,RET_strategy,RET_stock,RET_momentum
0,2011-01-31,100.376340,100.370025,101.994118,101.338189
1,2011-02-28,102.506684,101.921464,106.497506,106.903320
2,2011-03-31,103.324981,104.428539,107.862416,110.400000
3,2011-04-30,105.998926,105.022450,109.505742,112.171099
4,2011-05-31,104.384746,105.382831,107.265702,109.445287
...,...,...,...,...,...
132,2022-01-31,179.770843,312.814137,341.072137,391.892185
133,2022-02-28,177.618764,326.878908,338.811843,400.823391
134,2022-03-31,179.140295,331.761814,344.765798,413.904890
135,2022-04-30,167.827966,362.046212,307.658285,380.181798


In [23]:
returns.set_index(['DATE'],inplace=True)

In [24]:
returns.describe()

,RET_etf,RET_strategy,RET_stock,RET_momentum
count,137.000000,137.000000,137.000000,137.000000
mean,129.265605,178.680949,194.310469,213.155268
std,25.041322,56.416297,80.762543,103.740244
min,88.576989,100.370025,82.278899,79.901693
25%,109.592004,133.169458,144.136939,154.864015
50%,121.682987,172.908404,173.326069,186.727982
75%,142.088593,216.508431,218.151198,232.409337
max,187.893329,385.827960,409.655761,515.752723


In [25]:
returns

,RET_etf,RET_strategy,RET_stock,RET_momentum
DATE,,,,
2011-01-31,100.376340,100.370025,101.994118,101.338189
2011-02-28,102.506684,101.921464,106.497506,106.903320
2011-03-31,103.324981,104.428539,107.862416,110.400000
2011-04-30,105.998926,105.022450,109.505742,112.171099
2011-05-31,104.384746,105.382831,107.265702,109.445287
...,...,...,...,...
2022-01-31,179.770843,312.814137,341.072137,391.892185
2022-02-28,177.618764,326.878908,338.811843,400.823391
2022-03-31,179.140295,331.761814,344.765798,413.904890


In [26]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_etf'],name="Benchmark - ETF", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_stock'],name="Benchmark - Stock", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_strategy'],name="Strategy with Compounding", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=portfolio_rets.index, y=portfolio_rets['RET_strategy'],name="Strategy - const. Investment", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_momentum'],name="Momentum", mode = 'lines'),
    secondary_y=False
)

fig.show()

In [27]:
### Assume 100% Weightage on ML and none on Momentum

In [28]:
final = 1 * port1 + 0* port2
stats_bf = final.describe()
stats_bf.loc['Sharpe'] = (stats_bf.loc['mean'] - RF)/stats_bf.loc['std']*(12**0.5)
se_bf = stats_bf['RET']['std']/np.sqrt(stats_bf['RET']['count'])
print(f"t-stats: {round(stats_bf['RET']['mean']/se_bf,2)}")
stats_bf

t-stats: 6.91


,RET
count,137.000000
mean,0.015380
std,0.026065
min,-0.043633
25%,-0.003428
50%,0.013178
75%,0.029129
max,0.099102
Sharpe,1.711741


In [29]:
#Dollar return
final = final.reset_index()
final['DATE'] = pd.to_datetime(final['DATE'])
momentum = momentum.reset_index()
momentum['DATE'] = pd.to_datetime(momentum['DATE'])
value1 = pd.merge(benchmark_etf,final,how='outer',on = 'DATE',suffixes=['_etf','_strategy'])
value2 = pd.merge(benchmark_s,momentum,how='outer',on = 'DATE',suffixes=['_stock','_momentum'])
value = pd.merge(value1,value2,how='outer',on = 'DATE')
value

,DATE,RET_etf,RET_strategy,RET_stock,index,RET_momentum
0,2011-01-31,0.003763,0.036453,0.019941,0.0,0.013382
1,2011-02-28,0.021224,-0.000857,0.044153,1.0,0.054916
2,2011-03-31,0.007983,0.004937,0.012816,2.0,0.032709
3,2011-04-30,0.025879,-0.004194,0.015235,3.0,0.016043
4,2011-05-31,-0.015228,0.000159,-0.020456,4.0,-0.024300
...,...,...,...,...,...,...
133,2022-02-28,-0.011971,0.018373,-0.006627,133.0,0.022790
134,2022-03-31,0.008566,0.048970,0.017573,134.0,0.032637
135,2022-04-30,-0.063148,0.045183,-0.107631,135.0,-0.081475
136,2022-05-31,0.000933,0.056072,-0.024747,136.0,0.004822


In [30]:
value = value.set_index('DATE')
returns = value*100
returns = returns.cumsum()
returns

,RET_etf,RET_strategy,RET_stock,index,RET_momentum
DATE,,,,,
2011-01-31,0.376340,3.645275,1.994118,0.0,1.338189
2011-02-28,2.498697,3.559536,6.409459,100.0,6.829831
2011-03-31,3.296983,4.053196,7.691094,300.0,10.100712
2011-04-30,5.884881,3.633839,9.214634,600.0,11.704968
2011-05-31,4.362054,3.649737,7.169042,1000.0,9.274920
...,...,...,...,...,...
2022-02-28,64.037247,195.681725,143.345296,891100.0,164.741929
2022-03-31,64.893875,200.578731,145.102600,904500.0,168.005585
2022-04-30,58.579087,205.097013,134.339493,918000.0,159.858039


In [31]:
portfolio_rets=pd.DataFrame()
portfolio_rets['RET_strategy']=returns['RET_strategy']
portfolio_rets['RET_strategy']+=100
portfolio_rets

,RET_strategy
DATE,
2011-01-31,103.645275
2011-02-28,103.559536
2011-03-31,104.053196
2011-04-30,103.633839
2011-05-31,103.649737
...,...
2022-02-28,295.681725
2022-03-31,300.578731
2022-04-30,305.097013


In [32]:
momentum_rets = pd.DataFrame()
momentum_rets['RET_momentum']=returns['RET_momentum']
momentum_rets['RET_momentum']+=100

In [33]:
returns = value*100
returns = ((1 + value).cumprod()) *100
returns = returns.reset_index()
# returns = returns.drop(['RET_strategy','RET_momentum'],axis=1)

In [34]:
returns.set_index(['DATE'],inplace=True)

In [35]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_etf'],name="Benchmark - ETF", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_stock'],name="Benchmark - Stock", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_strategy'],name="Compound Strategy", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=portfolio_rets.index, y=portfolio_rets['RET_strategy'],name="Strategy - No Compounding", mode = 'lines'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=returns.index, y=returns['RET_momentum'],name="Momentum", mode = 'lines'),
    secondary_y=False
)

fig.show()